In [13]:
import cv2
import tensorflow as tf
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [14]:
path = '/root/Desktop/vgg16/trainingSet/'

In [15]:
global params
params = {'path' : path,
          'batch_size' : 50,
          'input_size': (32,32),
          'encoded_size': 8,   
          'lrn_rate': 0.0001, 
          'epochs': 200}

In [16]:
def get_file_paths(path):
    paths = sorted([os.path.join(root, file)  for root, dirs, files in os.walk(path) for file in files])
    paths = shuffle(paths)
    return paths[:int(len(paths)*.8)], paths[int(len(paths)*.8):]  #trainpaths, testpaths

In [17]:
def load_data(path):
    img = cv2.imread(path, 0)
#     if img is None: return (False, None)
    img = cv2.resize(cv2.threshold(img, 240, 1, cv2.THRESH_BINARY)[1], 
                     params['input_size'], # <-- input_size tuple of format (h,w)
                     cv2.INTER_AREA)
    return np.expand_dims(img, -1)

In [18]:
global conv, convt, maxpool, upsample, Relu
conv = tf.layers.conv2d
convt = tf.layers.conv2d_transpose
maxpool = tf.layers.max_pooling2d
upsample = tf.image.resize_nearest_neighbor
Relu = tf.nn.relu

In [19]:
def encoder(input):
    with tf.variable_scope('encoder') as scope:
        net = conv(input, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='Save_C1_1')
        net = conv(net, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='Save_C1_2')
        net = conv(net, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='Save_C1_3')
        net = maxpool(net, pool_size=2, padding='SAME', strides=2, name='M1')

        net = conv(net, filters=64, kernel_size=3, padding='SAME',  strides=1, activation=Relu, name='Save_C2_1')
        net = conv(net, filters=64, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='Save_C2_2')
        net = conv(net, filters=64, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='Save_C2_3')
        net = maxpool(net, pool_size=2, padding='SAME', strides=2, name='M2')

        net = conv(net, filters=64, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='C3_1')
        net = conv(net, filters=64, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='C3_2')
        encoded = conv(net, filters=64, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='C3_3')
    return encoded

In [20]:
def decoder(encoded):
    with tf.variable_scope('decoder') as scope:
        net = upsample(encoded, size=(16,16))
        net = convt(net, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='TrC1_1')
        net = convt(net, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='TrC1_2')
        net = convt(net, filters=32, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='TrC1_3')

        net = upsample(net, size=(32,32))
        decoded = convt(tf.layers.batch_normalization(net), filters=1, kernel_size=3, padding='SAME', strides=1, activation=Relu, name='TrC2')
    return decoded

In [21]:
def loss_function(real, gen):
    with tf.variable_scope('L2_loss') as scope:
        loss = tf.reduce_mean(tf.square(real - gen))
    return loss

In [22]:
def Autoencoder_Model():

    batchsize = params['batch_size']
    
    X = tf.placeholder(dtype = tf.float32, shape = [batchsize, 32, 32, 1])
    dec = decoder(encoder(X))
    loss = loss_function(X, dec)
    optimiz = tf.train.AdadeltaOptimizer(params['lrn_rate']).minimize(loss)

    init = tf.global_variables_initializer()
    sess.run(init)
    
    tf.summary.scalar('Loss_Value',loss)
    real = tf.summary.image("Input", X, max_outputs=2)
    generated = tf.summary.image("Output", dec, max_outputs=2)
    print('Setting up summary op...')
    summary_op = tf.summary.merge_all()
   
    print("Setting up Saver...") 
    saver = tf.train.Saver() 
    train_summary_writer = tf.summary.FileWriter('./log_dirAE/train', sess.graph)
    test_summary_writer = tf.summary.FileWriter('./log_dirAE/test', sess.graph)
    
    train_paths, test_paths = get_file_paths(params['path'])
    
    dataset_size = len(train_paths)
    n_batches = dataset_size//batchsize
    print("Dataset Size %d \nNo of Batches %d " %(dataset_size,n_batches))
    batch_data, test_batch_data = [], []
    
    counter = 1
    for epoch in range(params['epochs']):

        for idx in range(n_batches): 
            
            batch_paths = train_paths[idx * batchsize: (idx+1) * batchsize]
            batch_data = np.array([load_data(path) for path in batch_paths])
#             print("Train Batch shape ", str(batch_data.shape)) 
  
            _, train_loss, train_summary_str = sess.run([optimiz, loss, summary_op], 
                                                  feed_dict= {X: batch_data})
            train_summary_writer.add_summary(train_summary_str, counter)
            
            if idx % 10 == 0:
                print("Epoch %d \t Batch %d \t Loss %.4f"%(epoch, idx+1, train_loss))
                
                #Choosing random n=batchsize paths from 'test_paths' list
                test_batch_paths = np.array(test_paths)[np.random.choice(len(test_paths), batchsize)]
                test_batch_data = np.array([load_data(path) for path in test_batch_paths])
#                 print("Test Batch shape ", str(test_batch_data.shape))
                _, test_summary_str = sess.run([dec, summary_op], 
                                               feed_dict= {X: test_batch_data})
                
                test_summary_writer.add_summary(test_summary_str, counter)
            if counter %100 == 0:
                saver.save(sess, './log_dir_bangla_recov_weights/' + "model.ckpt", counter)
            
            counter += 1

In [23]:
from tensorflow.python.framework import ops

In [ ]:
ops.reset_default_graph()
global sess
config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()
# params['lrn_rate'] = 0.0009
Autoencoder_Model()

sess.close()

Setting up summary op...
Setting up Saver...
Dataset Size 33600 
No of Batches 672 
Epoch 0 	 Batch 1 	 Loss 0.0641
Epoch 0 	 Batch 11 	 Loss 0.0608
Epoch 0 	 Batch 21 	 Loss 0.0614
Epoch 0 	 Batch 31 	 Loss 0.0609
Epoch 0 	 Batch 41 	 Loss 0.0627
Epoch 0 	 Batch 51 	 Loss 0.0667
Epoch 0 	 Batch 61 	 Loss 0.0654
Epoch 0 	 Batch 71 	 Loss 0.0601
Epoch 0 	 Batch 81 	 Loss 0.0637
Epoch 0 	 Batch 91 	 Loss 0.0535
Epoch 0 	 Batch 101 	 Loss 0.0673
Epoch 0 	 Batch 111 	 Loss 0.0645
Epoch 0 	 Batch 121 	 Loss 0.0685
Epoch 0 	 Batch 131 	 Loss 0.0709
Epoch 0 	 Batch 141 	 Loss 0.0641
Epoch 0 	 Batch 151 	 Loss 0.0566
Epoch 0 	 Batch 161 	 Loss 0.0601
Epoch 0 	 Batch 171 	 Loss 0.0677
Epoch 0 	 Batch 181 	 Loss 0.0603
Epoch 0 	 Batch 191 	 Loss 0.0560
Epoch 0 	 Batch 201 	 Loss 0.0640
Epoch 0 	 Batch 211 	 Loss 0.0623
Epoch 0 	 Batch 221 	 Loss 0.0588
Epoch 0 	 Batch 231 	 Loss 0.0590
Epoch 0 	 Batch 241 	 Loss 0.0636
Epoch 0 	 Batch 251 	 Loss 0.0690
Epoch 0 	 Batch 261 	 Loss 0.0700
Epoch 0 	

Epoch 3 	 Batch 361 	 Loss 0.0677
Epoch 3 	 Batch 371 	 Loss 0.0664
Epoch 3 	 Batch 381 	 Loss 0.0608
Epoch 3 	 Batch 391 	 Loss 0.0582
Epoch 3 	 Batch 401 	 Loss 0.0637
Epoch 3 	 Batch 411 	 Loss 0.0703
Epoch 3 	 Batch 421 	 Loss 0.0684
Epoch 3 	 Batch 431 	 Loss 0.0650
Epoch 3 	 Batch 441 	 Loss 0.0662
Epoch 3 	 Batch 451 	 Loss 0.0660
Epoch 3 	 Batch 461 	 Loss 0.0664
Epoch 3 	 Batch 471 	 Loss 0.0611
Epoch 3 	 Batch 481 	 Loss 0.0717
Epoch 3 	 Batch 491 	 Loss 0.0656
Epoch 3 	 Batch 501 	 Loss 0.0763
Epoch 3 	 Batch 511 	 Loss 0.0658
Epoch 3 	 Batch 521 	 Loss 0.0711
Epoch 3 	 Batch 531 	 Loss 0.0618
Epoch 3 	 Batch 541 	 Loss 0.0688
Epoch 3 	 Batch 551 	 Loss 0.0576
Epoch 3 	 Batch 561 	 Loss 0.0602
Epoch 3 	 Batch 571 	 Loss 0.0661
Epoch 3 	 Batch 581 	 Loss 0.0637
Epoch 3 	 Batch 591 	 Loss 0.0638
Epoch 3 	 Batch 601 	 Loss 0.0585
Epoch 3 	 Batch 611 	 Loss 0.0551
Epoch 3 	 Batch 621 	 Loss 0.0564
Epoch 3 	 Batch 631 	 Loss 0.0606
Epoch 3 	 Batch 641 	 Loss 0.0676
Epoch 3 	 Batc

Epoch 7 	 Batch 71 	 Loss 0.0586
Epoch 7 	 Batch 81 	 Loss 0.0622
Epoch 7 	 Batch 91 	 Loss 0.0522
Epoch 7 	 Batch 101 	 Loss 0.0657
Epoch 7 	 Batch 111 	 Loss 0.0630
Epoch 7 	 Batch 121 	 Loss 0.0669
Epoch 7 	 Batch 131 	 Loss 0.0692
Epoch 7 	 Batch 141 	 Loss 0.0626
Epoch 7 	 Batch 151 	 Loss 0.0553
Epoch 7 	 Batch 161 	 Loss 0.0587
Epoch 7 	 Batch 171 	 Loss 0.0661
Epoch 7 	 Batch 181 	 Loss 0.0588
Epoch 7 	 Batch 191 	 Loss 0.0547
Epoch 7 	 Batch 201 	 Loss 0.0624
Epoch 7 	 Batch 211 	 Loss 0.0608
Epoch 7 	 Batch 221 	 Loss 0.0574
Epoch 7 	 Batch 231 	 Loss 0.0576
Epoch 7 	 Batch 241 	 Loss 0.0620
Epoch 7 	 Batch 251 	 Loss 0.0673
Epoch 7 	 Batch 261 	 Loss 0.0683
Epoch 7 	 Batch 271 	 Loss 0.0588
Epoch 7 	 Batch 281 	 Loss 0.0662
Epoch 7 	 Batch 291 	 Loss 0.0613
Epoch 7 	 Batch 301 	 Loss 0.0612
Epoch 7 	 Batch 311 	 Loss 0.0603
Epoch 7 	 Batch 321 	 Loss 0.0629
Epoch 7 	 Batch 331 	 Loss 0.0640
Epoch 7 	 Batch 341 	 Loss 0.0597
Epoch 7 	 Batch 351 	 Loss 0.0657
Epoch 7 	 Batch 3

Epoch 10 	 Batch 441 	 Loss 0.0642
Epoch 10 	 Batch 451 	 Loss 0.0640
Epoch 10 	 Batch 461 	 Loss 0.0644
Epoch 10 	 Batch 471 	 Loss 0.0593
Epoch 10 	 Batch 481 	 Loss 0.0695
Epoch 10 	 Batch 491 	 Loss 0.0636
Epoch 10 	 Batch 501 	 Loss 0.0739
Epoch 10 	 Batch 511 	 Loss 0.0638
Epoch 10 	 Batch 521 	 Loss 0.0689
Epoch 10 	 Batch 531 	 Loss 0.0600
Epoch 10 	 Batch 541 	 Loss 0.0667
Epoch 10 	 Batch 551 	 Loss 0.0559
Epoch 10 	 Batch 561 	 Loss 0.0584
Epoch 10 	 Batch 571 	 Loss 0.0641
Epoch 10 	 Batch 581 	 Loss 0.0618
Epoch 10 	 Batch 591 	 Loss 0.0619
Epoch 10 	 Batch 601 	 Loss 0.0568
Epoch 10 	 Batch 611 	 Loss 0.0535
Epoch 10 	 Batch 621 	 Loss 0.0548
Epoch 10 	 Batch 631 	 Loss 0.0588
Epoch 10 	 Batch 641 	 Loss 0.0655
Epoch 10 	 Batch 651 	 Loss 0.0559
Epoch 10 	 Batch 661 	 Loss 0.0609
Epoch 10 	 Batch 671 	 Loss 0.0536
Epoch 11 	 Batch 1 	 Loss 0.0614
Epoch 11 	 Batch 11 	 Loss 0.0583
Epoch 11 	 Batch 21 	 Loss 0.0589
Epoch 11 	 Batch 31 	 Loss 0.0584
Epoch 11 	 Batch 41 	 Los